In [76]:
import pandas as pd

In [77]:
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv

In [78]:
dataset = pd.read_csv('output_data/time_series/total-daily-tbl.csv')

In [79]:
dataset.head()

,Date_Confirmed,Region,total,daily
0,15-02-2020,China,2073.0,62.0
1,15-02-2020,Italy,3.0,0.0
2,15-02-2020,Spain,2.0,0.0
3,15-02-2020,United Kingdom,1.0,0.0
4,15-02-2020,"Korea, South",28.0,0.0


In [80]:
# Plot with plotly

# An example of building up the structure of a figure as a Python dictionary
# and then constructing a graph object figure from that dictionary.

In [81]:
years = list(data['Date_Confirmed'].unique())

In [82]:
# Make list of continents
regions = []
for regions in data['Region']:
    if regions not in regions:
        regions.append(regions)

In [83]:
# Make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

In [84]:
# Fill in most of layout
fig_dict['layout']['xaxis'] = {'title': 'casos totales', 'type': 'log'}
fig_dict['layout']['yaxis'] = {'title': 'casos diarios', 'type': 'log'}
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '15-02-2020',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
    }

fig_dict['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 300,
                                                                    'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Fecha:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [93]:
# Make data
year = '15-02-2020'
for continent in continents:
    dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
    dataset_by_year_and_cont = dataset_by_year[
        dataset_by_year['Region'] == continent]

In [50]:
dataset

,Date_Confirmed,Region,total,daily
0,15-02-2020,China,2073.0,62.0
1,15-02-2020,Italy,3.0,0.0
2,15-02-2020,Spain,2.0,0.0
3,15-02-2020,United Kingdom,1.0,0.0
4,15-02-2020,"Korea, South",28.0,0.0
...,...,...,...,...
1322,18-04-2020,TLAXCALA,70.0,10.0
1323,18-04-2020,VERACRUZ,68.0,9.0
1324,18-04-2020,YUCATAN,213.0,13.0
1325,18-04-2020,ZACATECAS,28.0,5.0


In [135]:

tbl.reset_index()

,Date_Confirmed,Region,total,daily
0,15-02-2020,"Korea, South",28.0,0.0
1,15-02-2020,US,13.0,0.0
2,16-02-2020,"Korea, South",29.0,1.0
3,16-02-2020,US,13.0,0.0
4,17-02-2020,"Korea, South",30.0,1.0
...,...,...,...,...
174,17-04-2020,"Korea, South",10635.0,22.0
175,17-04-2020,US,699706.0,32114.0
176,18-04-2020,Mexico,6875.0,578.0
177,18-04-2020,"Korea, South",10653.0,18.0
